## Introducing JuliaLang

While Python is the standard language in Google Colab., JuliaLang, which is faster and as flexible as Python, will be a primary choice in the exercise of the quantum many-body problems.

First of all, please execute the instruction of the following cell.

Type shitf+enter or use the button on the right-hand side of the cell.

The procedure to run JuliaLang on Google Colab., which was originally introduced by a developer in Google, is explained in the following articles written in Japanese:

https://qiita.com/ueuema/items/ca1b326f5df10a4203bd

https://qiita.com/cometscome_phys/items/1ba6ec181bb0fe1b35d5

In [ ]:
!curl -sSL "https://julialang-s3.julialang.org/bin/linux/x64/1.7/julia-1.7.2-linux-x86_64.tar.gz" -o julia.tar.gz
!tar -xzf julia.tar.gz -C /usr --strip-components 1
!rm -rf julia.tar.gz*
!julia -e 'using Pkg; Pkg.add("IJulia")'

## Settings of runtime

After executing the instructions in the above cell,

please check whether the runtime is "julia-1.7" or not. 

From the "runtime" in the menue, you can change the runtime.

When the runtime is "julia-1.7," please preserve the settings.

## Examining whether JuliaLang is introduced successfully or not

Please execute the instruction in the following cell.

**If you find the version information of JuliaLang, you introduced Julia successfully**

In [ ]:
versioninfo()

# An example of numerical simulations by JuliaLang

Then, let's run an example of a Julia code.

# Hubbard model

2nd quantization for electrons:

$|\Phi\rangle = \sum_{\nu_{j} = 0,1}C_{\nu_0 \nu_1 \nu_2 \nu_3 \cdots
\nu_{2\ell}\nu_{2\ell+1}
\cdots
\nu_{2L-2}\nu_{2L-1}
}(\hat{c}^{\dagger}_{0\uparrow})^{\nu_0}
(\hat{c}^{\dagger}_{0\downarrow})^{\nu_1}
(\hat{c}^{\dagger}_{1\uparrow})^{\nu_2}
(\hat{c}^{\dagger}_{1\downarrow})^{\nu_3}
\cdots
(\hat{c}^{\dagger}_{\ell \uparrow})^{\nu_{2\ell }}
(\hat{c}^{\dagger}_{\ell \downarrow})^{\nu_{2\ell +1}}
\cdots
(\hat{c}^{\dagger}_{L-1 \uparrow})^{\nu_{2L -2 }}
(\hat{c}^{\dagger}_{L-1 \downarrow})^{\nu_{2L -1}}
|0\rangle$

$\rightarrow$

$v\left[1+\sum_{j=0}^{2L-1} \nu_j \cdot 2^j \right] =  C_{\nu_0 \nu_1 \nu_2 \nu_3 \cdots
\nu_{2\ell}\nu_{2\ell+1}\cdots
\nu_{2L-2}\nu_{2L-1}}$

In [2]:
module Hubbard1D
#
  mutable struct Param
    t::Float64
    U::Float64
    L::Int64
    Param() = new()
  end
#
  function initialize(param::Param, t, U, L)
    param.t = t
    param.U = U
    param.L = L
  end
#
function parity(m)
    m ⊻= m>>1 # XOR
    m ⊻= m>>2 # XOR
    m = (m&Int64(0x1111111111111111) ) *  Int64(0x1111111111111111)
    return (m>>60)&1
end
#
function Cop(ell,i,c0)
  icomb = 2^ell
  if parity(i&icomb) == 1
    return i,0.0
  else
    j = i⊻icomb
    sgn = 1.0 - 2.0*parity(i&(icomb-1))
    return j, c0 * sgn
  end
end
#
function Aop(ell,i,c0)
  icomb = 2^ell
  if parity(i&icomb) == 0
    return i,0.0
  else
    j = i⊻icomb
    sgn = 1.0 - 2.0*parity(i&(icomb-1))
    return j, c0 * sgn
  end
end
# Hubbard hamiltonian
  function multiply(param::Param,v0,v1)
    for k = 1:2^(2*param.L)
        c0 = v0[k]
        i = k - 1
        # define 1D Hubbard
        for ell = 0:param.L-1
          j1, c1 = Aop(2*ell,i,c0)
          j2, c2 = Cop(2*mod(ell+1,param.L),j1,c1)
          v1[j2+1] -= param.t * c2
          j1, c1 = Aop(2*ell,i,c0)
          j2, c2 = Cop(2*mod(ell-1,param.L),j1,c1)
          v1[j2+1] -= param.t * c2
          j1, c1 = Aop(2*ell+1,i,c0)
          j2, c2 = Cop(2*mod(ell+1,param.L)+1,j1,c1)
          v1[j2+1] -= param.t * c2
          j1, c1 = Aop(2*ell+1,i,c0)
          j2, c2 = Cop(2*mod(ell-1,param.L)+1,j1,c1)
          v1[j2+1] -= param.t * c2
          j1, c1 = Aop(2*ell,i,c0)
          j2, c2 = Cop(2*ell,j1,c1)
          j1, c1 = Aop(2*ell+1,j2,c2)
          j2, c2 = Cop(2*ell+1,j1,c1)
          v1[j2+1] += param.U * c2
        end
    end
    return 0
  end
#
  function countbit(i,length)
    j = 0
    for k = 0:length-1
      icomb = 2^k
      j += parity(i&icomb)
    end
    return j
  end
#
end

Main.Hubbard1D

## Exercise

In [3]:
using .Hubbard1D

### parity: whether the number of electrons is even (0) or odd (1).

In [4]:
Hubbard1D.parity(Int(0b1011)) # 0th up, 0th down, 1th down

1

### creation and annihilation operators

In [5]:
ell = 4
icomb = 2^ell
string(icomb, base=2)
i = Int(0b1101) # 0th up, 1th up, 1th down 
println("i=",i) # digit for "1101"
j,c1= Hubbard1D.Cop(1,i,1.0) # generate an down-spin electron at 0th site 
println(string(j, base=2),"  ",c1)
j,c1= Hubbard1D.Aop(2,i,1.0) # annihilate an up-spin electron at 1st site
string(j, base=2)

i=13
1111  -1.0


"1001"

### Two site Hubbard model

In [6]:
t = 0.5 # be careful about the boundary condition
U = 8.0
L = 2
nelec = 2
param = Hubbard1D.Param()
Hubbard1D.initialize(param,t,U,L)
v0 = zeros(Complex{Float64}, 2^(2*param.L))
#
Ndim = 0
println("basis")
for k = 1:2^(2*param.L)
  if Hubbard1D.countbit(k-1,2*param.L) == nelec
    global Ndim += 1
    println(Ndim,"  ",string(k-1,base=2))
  end
end
# construct the hamiltonian matrix
matrixH = zeros(Complex{Float64}, Ndim, Ndim)
jcount = 0
for k = 1:2^(2*param.L)
  if Hubbard1D.countbit(k-1,2*param.L) == nelec
    jcount +=1
    v0[k] = 1.0
    v1 = zeros(Complex{Float64}, 2^(2*param.L))
    Hubbard1D.multiply(param,v0,v1)
    v0[k] = 0.0
    icount = 0
    for m = 1:2^(2*param.L)
      if Hubbard1D.countbit(m-1,2*param.L) == nelec
        icount += 1
        matrixH[icount,jcount] += v1[m]
      end
    end
  end
end

basis
1  11
2  101
3  110
4  1001
5  1010
6  1100


Please confirm the following calculation:

$\langle0|
(\hat{c}^{\ }_{L-1 \downarrow})^{\nu'_{2L -1}}
(\hat{c}^{\ }_{L-1 \uparrow})^{\nu'_{2L -2 }}
\cdots
(\hat{c}^{\ }_{1\downarrow})^{\nu'_3}
(\hat{c}^{\ }_{1\uparrow})^{\nu'_2}
(\hat{c}^{\ }_{0\downarrow})^{\nu'_1}
(\hat{c}^{\ }_{0\uparrow})^{\nu'_0}
$

$\times
(\hat{c}^{\dagger}_{0\uparrow})^{\nu_0}
(\hat{c}^{\dagger}_{0\downarrow})^{\nu_1}
(\hat{c}^{\dagger}_{1\uparrow})^{\nu_2}
(\hat{c}^{\dagger}_{1\downarrow})^{\nu_3}
\cdots
(\hat{c}^{\dagger}_{L-1 \uparrow})^{\nu_{2L -2 }}
(\hat{c}^{\dagger}_{L-1 \downarrow})^{\nu_{2L -1}}
|0\rangle$

$= \delta_{\nu_0,\nu'_0}
\delta_{\nu_1,\nu'_1}
\delta_{\nu_2,\nu'_2}
\delta_{\nu_3,\nu'_3}
\cdots
\delta_{\nu_{2L-2},\nu'_{2L-2}}
\delta_{\nu_{2L-1},\nu'_{2L-1}}$

diagonalize the hamiltonian matrix

In [7]:
using LinearAlgebra
eig = eigvals(matrixH)
wf = eigvecs(matrixH)
println("E_0=",eig[1])
wf[:,1]

E_0=-0.4721359549995794


6-element Vector{ComplexF64}:
 0.16245984811645325 + 0.0im
                 0.0 + 0.0im
 -0.6881909602355867 + 0.0im
  0.6881909602355867 + 0.0im
                 0.0 + 0.0im
 0.16245984811645298 - 0.0im